In [ ]:
import os
import geopandas as gp
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from pylab import *
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
plt.rcParams['axes.unicode_minus']=False#负号
plt.rcParams['font.sans-serif']=['Arial']###############
plt.rcParams['font.size'] = '7' # 设置字体大小 = '16' # 设置字体大小
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内

In [ ]:
# latlon_1degree_SJ_last=path+'/global_grid.shp'
version='v16_250104'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)
excel_dir=os.path.join(output_dir,f'stastics_excel')

# gdb=os.path.join(output_dir,f'7_correct_temp_file.gdb')
gdb=os.path.join(output_dir,f'8_compare.gdb')
fig_path=r'D:\lakemapping\10_paper_writting\figure\v8_250410'
os.makedirs(fig_path,exist_ok=True)

global_diss=r'D:\lakemapping\0_auxiliary_data\World_Continents\world_vector\World_continents_dissolve.shp'
Arid_Region=r'D:\lakemapping\0_auxiliary_data\AI\Global-AI_ET0_annual_v3\Global-AI_ET0_v3_annual\Global_AI_Reclassify_Clip_simple.shp'

color_list=['#B8DBB3','#72B063','#E29135','#94C6CD','#719AAC','#4A5F7E','#9e003a']#浅绿，深绿，橙色，浅蓝，中蓝，深蓝,红
label_list=['0.005~0.01 ${km^2}$','0.01~0.03 ${km^2}$','0.03~0.1 ${km^2}$','0.1~1 ${km^2}$','>1 ${km^2}$']
sub_fig_name=['a','b','c','d','e','f']
red_color_list=['#311d3f','#522546','#88304e']
# lake_category_bins=[0.005,0.01,1,100,10000,1000000]
# lake_category_labels=['Tiny','Small','Medium','Large','Super large']

lake_category_bins=[0.005,0.01,1,100,10000,1000000]
lake_category_labels=['1','3','10','100']

In [ ]:
latlon_1degree_SJ_last_data=gp.read_file(gdb,layer='GLAKESplus_PLD_SJ')
latlon_1degree_SJ_last_data

In [ ]:
Lat_data = latlon_1degree_SJ_last_data.drop(columns='geometry').groupby(['Lat_down']).sum().reset_index()
latitude= Lat_data['Lat_down'].values
Lat_data

In [ ]:
Lon_data = latlon_1degree_SJ_last_data.drop(columns='geometry').groupby(['Lon_left']).sum().reset_index()
longitude= Lon_data['Lon_left'].values
Lon_data

In [ ]:
Lon_data['hu_a1'].max()
Lon_data['hu_a10'].max()

In [ ]:
def add_background_fig(ax,ax_legend,data,terrain_map,column,legend_kwds,norm,grid_flag=True):
    map1 = Basemap( llcrnrlat=-60, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180,ax=ax)
    map1.readshapefile(global_diss[:-4],'global_diss',drawbounds=True,color='k',linewidth=0.4)
    
    tu=data.plot(
            ax=ax,
            cax=ax_legend,
            zorder=1,
            legend=True,  # 图例
            column=column,  # 分级设色字段#picover0\pia_0
            cmap=terrain_map,#terrain_map,  # 渐变色带的名称#Set2  #seismic
            legend_kwds=legend_kwds,
            norm=norm
            )
    
def draw_line(ax,xlim,xticks,ylim,yticks,line_color,label,x,y):
    ax.set_xlim(xlim)
    ax.set_xticks(xticks)
    ax.set_ylim(ylim)
    ax.set_yticks(yticks)
    ax.plot(x, y, color=line_color, linestyle = "-",linewidth = 0.8,label=label)
    
    
def custom_ticks(x, pos):
        if x == 0:
            return "0"
        poww=int(math.log10(x))
        return f"10$^{poww}$"

In [ ]:
columns=['diff_a10','diff_c10']
grid_labels=[[0,0,1,0],[0,0,0,1]]
titles=['The area difference (km$^2$) of lakes \nwith sizes between 0.01 and 0.1 km$^2$','The count difference of lakes \nwith sizes between 0.01 and 0.1 km$^2$']
# titles=['Difference of total lake area (km$^2$) for lakes between 0.01 and 0.1 km$^2$','Difference of lakes count for lakes between 0.01 and 0.1 km$^2$']
RdYlBu_r = plt.colormaps.get_cmap('RdBu_r')#
newcolors = RdYlBu_r(np.linspace(0,1, 256))
terrain_map = ListedColormap(newcolors)
    
b1=40
b2=1200
norms=[colors.Normalize(vmin=-b1, vmax=b1),colors.Normalize(vmin=-b2, vmax=b2)]
ticks=[np.arange(-b1,b1+1,10),np.arange(-b2,b2+1,400)]

In [ ]:
savepic=os.path.join(fig_path,'Figure9 c.png')
fig=plt.figure(figsize=(6,5.1),dpi=300)
gs = gridspec.GridSpec(2,1,height_ratios=[1,1],hspace =0.1)
ax2=plt.subplot(gs[0, 0],)
### b.格网面积
ax22 =ax2.inset_axes((0.45, -0.02,0.35,0.04)) 
ax2.text(-195,65,sub_fig_name[2],fontsize=12,ha='left',va='bottom',weight='bold')
for spine in ax2.spines.values():
    spine.set_visible(False)
legend_kwds={'orientation': "horizontal",'extend':'both','ticks': ticks[0]}
add_background_fig(ax2,ax22,latlon_1degree_SJ_last_data,terrain_map,columns[0],legend_kwds,norms[0])
ax22.text(0,1.2, titles[0],bbox=dict(facecolor='white',edgecolor='White',pad=0),ha='center',va='bottom')#
for label in ax22.get_xticklabels():
        label.set_fontsize(6)
        
### b.格网面积
ax4=plt.subplot(gs[1, 0],)
for spine in ax4.spines.values():
        spine.set_visible(False)
legend_kwds={'orientation': "horizontal",'extend':'both','ticks': ticks[1]}
ax44 = ax4.inset_axes((0.45, -0.02,0.35,0.04)) 
ax4.text(-195,65,sub_fig_name[4],fontsize=12,ha='left',va='bottom',weight='bold')
add_background_fig(ax4,ax44,latlon_1degree_SJ_last_data,terrain_map,columns[1],legend_kwds,norms[1])
ax44.text(0,1.2, titles[1],bbox=dict(facecolor='white',edgecolor='White',pad=0),ha='center',va='bottom')
for label in ax44.get_xticklabels():
        label.set_fontsize(6)
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
hu_color=color_list[-1]
alpha=[0.3,0.5,0.7]
color_gt10='#363636'

In [ ]:
### a.经度面积
label=['≥ 0.1 km$^2$','0.01~0.1 km$^2$','< 0.01 km$^2$']
savepic=os.path.join(fig_path,'Figure9 a.png')
fig=plt.figure(figsize=(6,1),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(2,1,height_ratios=[1,1],hspace =0)

ax1=plt.subplot(gs[0, 0],)
ax1.text(-290,2.8,sub_fig_name[0],fontsize=12,ha='left',weight='bold')

s=3
hu_a_lt1=Lon_data['hu_a1'].values/(10.0**s)
hu_a_bt1_10=Lon_data['hu_a10'].values/(10.0**s)
pld_a_bt1_10=Lon_data['pld_a10'].values/(10.0**s)

s1=ax1.bar(longitude+0.5,bottom=hu_a_bt1_10,height=hu_a_lt1, color=hu_color,width=1,alpha=alpha[2],label=label[2])
s2=ax1.bar(longitude+0.5,hu_a_bt1_10, color=hu_color,width=1,alpha=alpha[1],label=label[1])
# ax1.bar(longitude+0.5,pld_a_bt1_10,color='#719AAC',width=1,alpha=alpha[1],label=label[1])
s3,=ax1.plot(longitude+0.5,pld_a_bt1_10,color=red_color_list[1],linewidth=0.6,label=' ')#label[1]
plt.tick_params(bottom=True, top=True, left=True, right=False )
plt.tick_params(labelbottom=False,labeltop=False,labelleft=True, labelright=False)
ax1.set_ylim(0,3)
ax1.set_xlim(-180,180)
ax1.text(-177,2,f'×10$^{s}$',ha='left',va='bottom',color=hu_color,fontsize=6)
ax1.set_yticks(range(0,4))
ax1.text(177,2,'area < 0.1 km$^2$',ha='right',va='bottom',color=color_gt10)
ax11=plt.subplot(gs[1, 0],)
s=4
hu_a_gt10=Lon_data['hu_a_gt10'].values/(10.0**s)
pld_a_gt10=Lon_data['pld_a_gt10'].values/(10.0**s)
s4=ax11.bar(longitude+0.5,-hu_a_gt10, color=hu_color,width=1,alpha=alpha[0],label=label[0])
# ax11.bar(longitude+0.5,-pld_a_gt10,color=color_list[-1],width=1,alpha=alpha[0],label=label[0])
s5,=ax11.plot(longitude+0.5,-pld_a_gt10,color=red_color_list[2],linewidth=0.6,label=' ')
ax11.set_ylim(-6,0)
ax11.text(-200,0,'Area ($\mathregular{km^2}$)',rotation=90,va='center',color=hu_color)

for ax in [ax1,ax11]:
    ax.set_xlim(-180,180)
    ax.set_xticks(range(-180,181,60))
    ax.spines['left'].set_color(hu_color) 
    ax.tick_params(axis='x',width=0.5,length=2)
    ax.tick_params(axis='y',width=0.5,length=2,color=hu_color)
    for label in ax.get_yticklabels():
        label.set_color(hu_color)
        label.set_fontsize(6)
    for label in ax.get_xticklabels():
        label.set_fontsize(6)
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)

plt.tick_params(bottom=True, top=True, left=True, right=False )
plt.tick_params(labelbottom=True,labeltop=False,labelleft=True, labelright=False)
series_hu=[s4,s2,s1]
series_pld=[s5,s3]
ax11.set_yticks(range(-6,0,2))
ax11.set_yticklabels([6,4,2])
# ax1.text(-272,2.1,'Size groups',fontsize=7)
# ax1.text(-285,1.3,'PLD  GLAKESplus',fontsize=6,color=hu_color)
ax1.text(-272,1.4,'Size groups',fontsize=7)
ax1.text(-278,-2.7,'PLD  GLAKESplus',fontsize=6,color=hu_color)
ax11.set_xlabel('Longitude')
ax1.legend(handles=series_hu,title=None,handlelength=2,labelspacing=0.5,loc=[-0.23,-0.75],fontsize=6,frameon=False)
ax11.legend(loc=[1.02,0.1],frameon=False)
ax11.legend(handles=series_pld, title=None,handlelength=2,labelspacing=0.59,loc=[-0.28,0.61],fontsize=6, frameon=False)
ax11.set_facecolor('#f4f4f4')
ax11.text(-177,-6,f'×10$^{s}$',ha='left',va='bottom', color=hu_color,fontsize=6)
ax11.text(177,-6,'area ≥ 0.1 km$^2$',ha='right',va='bottom',color=color_gt10)

# with PdfPages(savepic.replace('eps','pdf')) as pdf:
#     pdf.savefig(fig)

# # 保存为.ps格式
# plt.savefig(savepic)
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
# plt.show()

In [ ]:
hu_c_color=color_list[-2]
label=['≥ 0.1 km$^2$','0.01~0.1 km$^2$','< 0.01 km$^2$']
savepic=os.path.join(fig_path,'Figure9 e.png')
fig=plt.figure(figsize=(6,1),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(2,1,height_ratios=[1,1],hspace =0)

ax1=plt.subplot(gs[0, 0],)
ax1.text(-290,13,sub_fig_name[5],fontsize=12,ha='left',weight='bold')

s=4
hu_a_lt1=Lon_data['hu_c1'].values/(10.0**s)
hu_a_bt1_10=Lon_data['hu_c10'].values/(10.0**s)
pld_a_bt1_10=Lon_data['pld_c10'].values/(10.0**s)

s1=ax1.bar(longitude+0.5,bottom=hu_a_bt1_10,height=hu_a_lt1, color=hu_c_color,width=1,alpha=alpha[2],label=label[2])
s2=ax1.bar(longitude+0.5,hu_a_bt1_10, color=hu_c_color,width=1,alpha=alpha[1],label=label[1])
# ax1.bar(longitude+0.5,pld_a_bt1_10,color='#719AAC',width=1,alpha=alpha[1],label=label[1])
s3,=ax1.plot(longitude+0.5,pld_a_bt1_10,color='k',linewidth=0.6,label=' ')
plt.tick_params(bottom=True, top=True, left=True, right=False )
plt.tick_params(labelbottom=False,labeltop=False,labelleft=True, labelright=False)
ax1.set_ylim(0,15)
ax1.set_xlim(-180,180)
ax1.text(-177,10,f'×10$^{s}$',ha='left',va='bottom',color=hu_c_color,fontsize=6)
# ax1.set_ylabel('Count')
ax1.set_yticks(range(0,16,5))
ax1.text(177,10,'Area < 0.1 km$^2$',ha='right',va='bottom',color=color_gt10)
ax11=plt.subplot(gs[1, 0],)
s=4
hu_a_gt10=Lon_data['hu_c_gt10'].values/(10.0**s)
pld_a_gt10=Lon_data['pld_c_gt10'].values/(10.0**s)
s4=ax11.bar(longitude+0.5,-hu_a_gt10, color=hu_c_color,width=1,alpha=alpha[0],label=label[0])
# ax11.bar(longitude+0.5,-pld_a_gt10,color='#719AAC',width=1,alpha=alpha[0],label=label[0])
s5,=ax11.plot(longitude+0.5,-pld_a_gt10,color=color_list[-2],linewidth=0.6,label=' ')
ax11.set_ylim(-3,0)
for ax in [ax1,ax11]:
    ax.set_xlim(-180,180)
    ax.set_xticks(range(-180,181,60))
    ax.spines['left'].set_color(hu_c_color) 
    ax.tick_params(axis='x',width=0.5,length=2)
    ax.tick_params(axis='y',width=0.5,length=2,color=hu_c_color)
    for label in ax.get_yticklabels():
        label.set_color(hu_c_color)
        label.set_fontsize(6)
    for label in ax.get_xticklabels():
        label.set_fontsize(6)
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)

plt.tick_params(bottom=True, top=True, left=True, right=False )
plt.tick_params(labelbottom=True,labeltop=False,labelleft=True, labelright=False)

series_hu=[s4,s2,s1]
series_pld=[s5,s3]

ax11.set_yticks(range(-3,0,1))
ax11.set_yticklabels([3,2,1])
ax1.text(-272,6.5,'Size groups',fontsize=7)
ax1.text(-278,-14,'PLD  GLAKESplus',fontsize=6,color=hu_c_color)

ax1.legend(handles=series_hu,title=None,handlelength=2,labelspacing=0.5,loc=[-0.23,-0.8],fontsize=6,frameon=False)
# ax11.legend(loc=[1.02,0.1],frameon=False)
ax11.legend(handles=series_pld, title=None,handlelength=2,labelspacing=0.59,loc=[-0.28,0.55],fontsize=6, frameon=False)
ax11.text(-177,-3,f'×10$^{s}$',ha='left',va='bottom', color=hu_c_color,fontsize=6)
ax11.text(-200,0,'Count',rotation=90,va='center',color=hu_c_color)
ax11.set_facecolor('#f4f4f4')
ax11.text(177,-3,'Area ≥ 0.1 km$^2$',ha='right',va='bottom',color=color_gt10)
ax11.set_xlabel('Longitude')
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
### c.纬度面积
label=['>0.1 km$^2$','0.01~0.1 km$^2$','0.005~0.01 km$^2$']
savepic=os.path.join(fig_path,'Figure9 b.png')
fig=plt.figure(figsize=(1,2.5),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(1,2,width_ratios=[1,1],wspace =0)

ax1=plt.subplot(gs[0, 1],)
s=3
hu_a_lt1=Lat_data['hu_a1'].values/(10.0**s)
hu_a_bt1_10=Lat_data['hu_a10'].values/(10.0**s)
pld_a_bt1_10=Lat_data['pld_a10'].values/(10.0**s)

ax1.barh(latitude+0.5,left=hu_a_bt1_10,width=hu_a_lt1, color=hu_color,height=1,alpha=alpha[2],label=label[2])
ax1.barh(latitude+0.5,hu_a_bt1_10, color=hu_color,height=1,alpha=alpha[1],label=label[1])
# ax1.barh(latitude+0.5,pld_a_bt1_10,color=red_color[0],height=1,alpha=alpha[1],label=label[1])
ax1.plot(pld_a_bt1_10,latitude+0.5,color=red_color_list[1],linewidth=0.6,label=label[1])

plt.tick_params(bottom=False, top=True, left=True, right=True )
plt.tick_params(labelbottom=False,labeltop=True,labelleft=False, labelright=False)

ax1.set_xlim(0,12)
ax1.set_xticks(range(0,13,4))
ax1.text(11,85,f'×10$^{s}$',ha='right',va='top',color=hu_color,fontsize=6)

ax11=plt.subplot(gs[0, 0],)
ax11.text(-18,65,sub_fig_name[1],fontsize=12,ha='right',va='bottom',weight='bold')
# ax11.set_xlabel('Area ($\mathregular{km^2}$)')

s=4
hu_a_gt10=Lat_data['hu_a_gt10'].values/(10.0**s)
pld_a_gt10=Lat_data['pld_a_gt10'].values/(10.0**s)
ax11.barh(latitude+0.5,-hu_a_gt10, color=hu_color,height=1,alpha=alpha[0],label=label[0])
# ax11.barh(latitude+0.5,-pld_a_gt10,color=red_color_list[1],height=1,alpha=alpha[0],label=label[0])
ax11.plot(-pld_a_gt10,latitude+0.5,color=red_color_list[2],linewidth=0.6,label=label[0])
ax11.set_xlim(-12,0)
ax11.set_xticks(range(-12,0,4))
ax11.set_xticklabels([12,8,4])

plt.tick_params(bottom=False, top=True, left=True, right=True )
plt.tick_params(labelbottom=False,labeltop=True,labelleft=True, labelright=False)

for ax in [ax1,ax11]:
    ax.set_ylim(-60,90)
    ax.set_yticks(range(-60,91,30))
    ax.spines['top'].set_color(hu_color) 
    ax.tick_params(axis='y',width=0.5,length=2)
    ax.tick_params(axis='x',top=True,width=0.5,length=2,color=hu_color)
    for label in ax.get_xticklabels():
        print(label)
        label.set_color(hu_color)
        label.set_fontsize(6)
    for label in ax.get_yticklabels():
        label.set_fontsize(6)
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)


ax11.set_facecolor('#f4f4f4')
ax11.text(0,102,'Area (km$^2$)',color=hu_color,ha='center',va='bottom')
ax11.text(-11,85,f'×10$^{s}$',ha='left',va='top', color=hu_color,fontsize=6)
ax11.set_ylabel('Latitude')
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
### e.纬度数量
label=['>0.1 km$^2$','0.01~0.1 km$^2$','0.005~0.01 km$^2$']
savepic=os.path.join(fig_path,'Figure9 d.png')
fig=plt.figure(figsize=(1,2.5),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(1,2,width_ratios=[1,1],wspace =0)

ax1=plt.subplot(gs[0, 1],)
s=5
hu_a_lt1=Lat_data['hu_c1'].values/(10.0**s)
hu_a_bt1_10=Lat_data['hu_c10'].values/(10.0**s)
pld_a_bt1_10=Lat_data['pld_c10'].values/(10.0**s)
s1=ax1.barh(latitude+0.5,left=hu_a_bt1_10,width=hu_a_lt1, color=hu_c_color,height=1,alpha=alpha[2],label=label[2])
s2=ax1.barh(latitude+0.5,hu_a_bt1_10, color=hu_c_color,height=1,alpha=alpha[1],label=label[1])
# ax1.barh(latitude+0.5,pld_a_bt1_10,color=red_color[0],height=1,alpha=alpha[1],label=label[1])
s4=ax1.plot(pld_a_bt1_10,latitude+0.5,color='k',linewidth=0.6,label=label[1])

plt.tick_params(bottom=True, top=False, left=True, right=True )
plt.tick_params(labelbottom=True,labeltop=False,labelleft=False, labelright=False)

ax1.set_xlim(0,5)
ax1.set_xticks(range(0,6,1))
ax1.text(4.5,-55,f'×10$^{s}$',ha='right',va='bottom',color=hu_c_color,fontsize=6)

ax1.text(2.5,91,'Area \n< 0.1 km$^2$',ha='center',va='bottom',color=color_gt10,fontsize=6)
ax11=plt.subplot(gs[0, 0],)
ax11.text(-13.5,65,sub_fig_name[3],fontsize=12,ha='right',va='bottom',weight='bold')
# ax11.set_xlabel('Count')

s=4
hu_a_gt10=Lat_data['hu_c_gt10'].values/(10.0**s)
pld_a_gt10=Lat_data['pld_c_gt10'].values/(10.0**s)
s3=ax11.barh(latitude+0.5,-hu_a_gt10, color=hu_c_color,height=1,alpha=alpha[0],label=label[0])
# ax11.barh(latitude+0.5,-pld_a_gt10,color=red_color_list[1],height=1,alpha=alpha[0],label=label[0])
s5=ax11.plot(-pld_a_gt10,latitude+0.5,color=color_list[-2],linewidth=0.6,label=label[0])
ax11.set_xlim(-9,0)
ax11.set_xticks(range(-9,0,3))
ax11.set_xticklabels([9,6,3])
for ax in [ax1,ax11]:
    ax.set_ylim(-60,90)
    ax.set_yticks(range(-60,91,30))
    ax.spines['bottom'].set_color(hu_c_color) 
    ax.tick_params(axis='y',width=0.5,length=2)
    ax.tick_params(axis='x',width=0.5,length=2,color=hu_c_color)
    for label in ax.get_xticklabels():
        label.set_color(hu_c_color)
        label.set_fontsize(6)
    for label in ax.get_yticklabels():
        label.set_fontsize(6)
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)

plt.tick_params(bottom=True, top=False, left=True, right=True )
plt.tick_params(labelbottom=True,labeltop=False,labelleft=True, labelright=False)

# ax11.set_xticklabels([])
ax11.set_facecolor('#f4f4f4')
# ax1.legend(loc=[1.02,0],frameon=False)
# ax11.legend(loc=[1.02,0.1],frameon=False)
ax11.set_ylabel('Latitude')
ax11.text(-8,-55,f'×10$^{s}$',ha='left',va='bottom', color=hu_c_color,fontsize=6)
ax11.text(0,-75,'Count',color=hu_c_color,ha='center')
ax11.text(-4.5,91,'Area \n≥ 0.1 km$^2$',ha='center',va='bottom',color=color_gt10,fontsize=6)
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()